In [1]:
import pymysql
# 引入python中的traceback模块，跟踪错误
import traceback
import sys


class MysqlUtil():
    def __init__(self):
        '''
            初始化方法，连接数据库
        '''
        host = '192.168.8.126'  # 主机名
        user = 'root'  # 数据库用户名
        password = '8008'  # 数据库密码
        database = 'weibo'  # 数据库名称
        self.db = pymysql.connect(host=host, user=user, password=password, db=database)  # 建立连接
        self.cursor = self.db.cursor(cursor=pymysql.cursors.DictCursor)  # 设置游标，并将游标设置为字典类型

    def insert(self, sql):
        '''
            插入数据库
            sql:插入数据库的sql语句
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            # 提交到数据库执行
            self.db.commit()
        except Exception:  # 方法一：捕获所有异常
            # 如果发生异常，则回滚
            print("发生异常", Exception)
            self.db.rollback()
        finally:
            # 最终关闭数据库连接
            self.db.close()

    def fetchone(self, sql):
        '''
            查询数据库：单个结果集
            fetchone(): 该方法获取下一个查询结果集。结果集是一个对象
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            result = self.cursor.fetchone()
        except:  # 方法二：采用traceback模块查看异常
            # 输出异常信息
            traceback.print_exc()
            # 如果发生异常，则回滚
            self.db.rollback()
            result = 'None'

        finally:
            # 最终关闭数据库连接
            self.db.close()
        return result

    def fetchall(self, sql):
        '''
            查询数据库：多个结果集
            fetchall(): 接收全部的返回结果行.
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            results = self.cursor.fetchall()
        except:  # 方法三：采用sys模块回溯最后的异常
            # 输出异常信息
            info = sys.exc_info()
            print(info[0], ":", info[1])
            # 如果发生异常，则回滚
            self.db.rollback()
            results = 'None'
        finally:
            # 最终关闭数据库连接
            self.db.close()
        return results

    def delete(self, sql):
        '''
            删除结果集
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            self.db.commit()
        except:  # 把这些异常保存到一个日志文件中，来分析这些异常
            # 将错误日志输入到目录文件中
            f = open("\log.txt", 'a')
            traceback.print_exc(file=f)
            f.flush()
            f.close()
            # 如果发生异常，则回滚
            self.db.rollback()
        finally:
            # 最终关闭数据库连接
            self.db.close()

    def update(self, sql):
        '''
            更新结果集
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            self.db.commit()
        except:
            # 如果发生异常，则回滚
            self.db.rollback()
        finally:
            # 最终关闭数据库连接
            self.db.close()


In [2]:
import requests
from urllib.parse import urlencode
import time
import random
from pyquery import PyQuery as pq
import sys

# 设置代理等（新浪微博的数据是用ajax异步下拉加载的，network->xhr）
host = 'm.weibo.cn'
base_url = 'https://%s/api/container/getIndex?' % host
user_agent = 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Mobile Safari/537.36'

# 设置请求头
headers = {
    'Host': host,
    'keep': 'close',
    # 'Referer': 'https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%A6%82%E4%BD%95%E7%9C%8B%E5%BE%85%E5%8F%8D%E5%86%85%E5%8D%B7%E7%83%AD%E6%BD%AE%23&extparam=%23%E5%A6%82%E4%BD%95%E7%9C%8B%E5%BE%85%E5%8F%8D%E5%86%85%E5%8D%B7%E7%83%AD%E6%BD%AE%23&luicode=10000011&lfid=100103type%3D38%26q%3D%E5%86%85%E5%8D%B7%26t%3D0',
    'User-Agent': user_agent
}

save_per_n_page = 10

from datetime import datetime


def time_formater(input_time_str):
    input_format = '%a %b %d %H:%M:%S %z %Y'
    output_format = '%Y-%m-%d %H:%M:%S'

    return datetime.strptime(input_time_str, input_format).strftime(output_format)


# 按页数抓取数据
def get_single_page(page, keyword):
    # 请求参数
    params = {
        'containerid': f'100103type=1&q=#{keyword}#',  # 、、教育内卷、职场内卷、如何看待内卷的社会状态、如何避免婚姻内卷、
        'page_type': 'searchall',
        'page': page
    }
    url = base_url + urlencode(params)
    print(url)
    error_times = 3
    while True:
        response = requests.get(url, headers=headers)  # ,proxies=abstract_ip.get_proxy()
        if response.status_code == 200:
            if len(response.json().get('data').get('cards')) > 0:
                return response.json()
        time.sleep(3)
        error_times += 1
        # 连续出错次数超过 3
        if error_times > 3:
            return None


# 长文本爬取代码段
def getLongText(lid):  # lid为长文本对应的id
    # 长文本请求头
    headers_longtext = {
        'Host': host,
        'Referer': 'https://m.weibo.cn/status/' + lid,
        'User-Agent': user_agent
    }
    params = {
        'id': lid
    }
    url = 'https://m.weibo.cn/statuses/extend?' + urlencode(params)
    try:
        response = requests.get(url, headers=headers_longtext)  # proxies=abstract_ip.get_proxy()
        if response.status_code == 200:  # 数据返回成功
            jsondata = response.json()
            tmp = jsondata.get('data')
            return pq(tmp.get("longTextContent")).text()  # 解析返回结构，获取长文本对应内容
    except:
        pass


# 解析页面返回的json数据
count = 0

'''
修改后的页面爬取解析函数
'''


def parse_page(json_data):
    global count
    items = json_data.get('data').get('cards')

    for index, item in enumerate(items):
        if item.get('card_type') == 7:
            print('导语')
            continue
        elif item.get('card_type') == 8 or (item.get('card_type') == 11 and item.get('card_group') is None):
            continue
        # topic = json_data.get('data').get('cardlistInfo').get('cardlist_head_cards')[0]
        # # 单独的关键词抓取不是超话，会有 topic == null
        # if topic is None or topic.get('head_data', None) is None:
        #     topic = keyword
        # else:
        #     topic = topic.get('head_data').get('title')
        if item.get('mblog', None):
            item = item.get('mblog')
        else:
            item = item.get('card_group')[0].get('mblog')
        if item:
            if item.get('isLongText') is False:  # 不是长文本
                data = {
                    'wid': item.get('id'),
                    'user_name': item.get('user').get('screen_name'),
                    'user_id': item.get('user').get('id'),
                    'gender': item.get('user').get('gender'),
                    'publish_time': time_formater(item.get('created_at')),
                    'text': pq(item.get("text")).text(),  # 仅提取内容中的文本
                    'like_count': item.get('attitudes_count'),  # 点赞数
                    'comment_count': item.get('comments_count'),  # 评论数
                    'forward_count': item.get('reposts_count'),  # 转发数
                }
            else:  # 长文本涉及文本的展开
                tmp = getLongText(item.get('id'))  # 调用函数
                data = {
                    'wid': item.get('id'),
                    'user_name': item.get('user').get('screen_name'),
                    'user_id': item.get('user').get('id'),
                    'gender': item.get('user').get('gender'),
                    'publish_time': time_formater(item.get('created_at')),
                    'text': tmp,  # 仅提取内容中的文本
                    'like_count': item.get('attitudes_count'),
                    'comment_count': item.get('comments_count'),
                    'forward_count': item.get('reposts_count'),
                }
            count += 1
            print(f'total count: {count}')
            yield data


import os, csv

if __name__ == '__main__':
    keyword = '张新成 受伤'
    keyword = sys.argv[1]
    print (keyword)
    sys.exit()
    result_file = f'{keyword}.csv'
    if not os.path.exists(result_file):
        with open(result_file, mode='w', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['wid', 'user_name', 'user_id', 'gender',
                             'publish_time', 'text', 'like_count', 'comment_count',
                             'forward_count'])

    temp_data = []

    empty_times = 0

    for page in range(1, 50000):  # 瀑布流下拉式，加载
        print(f'page: {page}')
        json_data = get_single_page(page, keyword)
        if json_data == None:
            print('json is none')
            break

        if len(json_data.get('data').get('cards')) <= 0:
            empty_times += 1
        else:
            empty_times = 0
        if empty_times > 3:
            print('\n\n consist empty over 3 times \n\n')
            break

        for result in parse_page(json_data):  # 需要存入的字段
            temp_data.append(result)
        if page % save_per_n_page == 0:
            with open(result_file, mode='a+', encoding='utf-8-sig', newline='') as f:
                writer = csv.writer(f)
                for d in temp_data:
                    writer.writerow(
                        [d['wid'], d['user_name'], d['user_id'], d['gender'],
                         d['publish_time'], d['text'], d['like_count'], d['comment_count'],
                         d['forward_count']])
            print(f'\n\n------cur turn write {len(temp_data)} rows to csv------\n\n')
            sys.exit()
            temp_data = []
        time.sleep(random.randint(2, 6))  # 爬取时间间隔


-f


SystemExit: 

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
import csv
import emoji
title='多平台回应刑满释放人员变励志网红'
with open(title+'.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    list_of_csv = list(csv_reader)

for kk in range(len(list_of_csv)-1):
    kk = kk + 1

    ss=list_of_csv[kk][5]
    import demoji
    demoji.download_codes()
    res = emoji.replace_emoji(ss, replace=" ")
    if res!=ss:
        for o in demoji.findall(ss).values():
            res=res+''
            res=res+o
        list_of_csv[kk][5]=res
    sql = "INSERT INTO weiboevents(wid,title,user_id,user_name,gender,publish_time,text,like_count,comment_count,forward_count) \
                       VALUES ('%s', '%s', '%s','%s','%s','%s','%s','%d','%d','%d')" % (list_of_csv[kk][0], title,list_of_csv[kk][2],list_of_csv[kk][1],list_of_csv[kk][3],list_of_csv[kk][4],list_of_csv[kk][5],int(list_of_csv[kk][6]),int(list_of_csv[kk][7]),int(list_of_csv[kk][8]))
    db = MysqlUtil()
    db.insert(sql)

/var/folders/3f/hj3gtg8n3gz5xm84jn7ywct40000gn/T/ipykernel_2288/3786547022.py:13: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [4]:
sql

"INSERT INTO weiboevents(wid,title,user_id,user_name,gender,publish_time,text,like_count,comment_count,forward_count)                        VALUES ('4858986648052869', '多平台回应刑满释放人员变励志网红', '1647688972','环球人物杂志','m','2023-01-17 19:49:15','【#环球人物评刑释人员当网红#：污点不是卖点，包容绝非纵容】\n近期，有媒体发现，“刑满释放”竟然被某些业内人士总结为短视频和直播的“新赛道”。真是资本无底线，万物都敢“赛道”。\n\n刑释人员可以当网红吗？法律并无明文限制，国家也鼓励他们回归社会。如果刑释人员真正找到了新的生活方式，重新实现了正向的人生价值，当然有权通过网络合法合规地分享其经历和感悟。这也是整个社会对刑释人员本着“宽严相济、治病救人”原则的体现。\n\n但是，如果刑释人员根本就没有走上洗心革面、自力更生之路，而是靠卖惨成了网红，靠晒“释放证明书”博人眼球，靠流量来走捷径“带货”，靠利用人们的同情心“躺赢”。这算哪门子励志？这有什么正能量？恰恰相反，平台大肆推送卖惨刑释人员的账号，是毫无底线，是流量资本的嗜血与围猎。为了流量，不惜违背“公序良俗”，不惜对受害者造成二次伤害。人们对刑释人员当网红一片哗然，原因就在此。\n\n污点不是卖点，包容绝非纵容。鼓励刑释人员寻找人生的多种可能性，绝不意味着可以把过去犯下的错误当作流量密码。“网红”已成为一种炙手可热的新职业，面对新的形势，法律法规如何细化？平台监管如何进行？社会和舆论如何引导帮助刑释人员重回正轨？诸多问题值得我们深思。\n#多平台回应刑满释放人员变励志网红##刑释人员可不可以当网红#','6','1','1')"

In [42]:
list_of_csv[1]

['4857921639556034',
 '张新成StevenZhang工作室',
 '7782851856',
 'm',
 '2023-01-14 21:17:17',
 '本哥在节目录制期间，意外手臂受伤，已于第一时间送医治疗。现已无大碍，请大家不要担心～',
 '10230',
 '1866',
 '985']

In [80]:
kk=163
sql = "INSERT INTO weiboevents(wid,title,user_id,user_name,gender,publish_time,text,like_count,comment_count,forward_count) \
                       VALUES ('%s', '%s', '%s','%s','%s','%s','%s','%d','%d','%d')" % (list_of_csv[kk][0], title,list_of_csv[kk][2],list_of_csv[kk][1],list_of_csv[kk][3],list_of_csv[kk][4],list_of_csv[kk][5],int(list_of_csv[kk][6]),int(list_of_csv[kk][7]),int(list_of_csv[kk][8]))


In [81]:
sql

"INSERT INTO weiboevents(wid,title,user_id,user_name,gender,publish_time,text,like_count,comment_count,forward_count)                        VALUES ('4857955029884528', '张新成 受伤', '6109698934','思马斯徂-','f','2023-01-14 23:29:58','#张新成 受伤#慢综 你没事儿吧😅😅😅','0','0','0')"

In [191]:
sql_4="select title,count(`id`) as t from weiboevents group by title"
db = MysqlUtil()
count_number = db.fetchall(sql_4)

In [194]:
count_number

[{'id': 1.0, 'title': '张新成 受伤', 't': 198}, {'id': 2.0, 'title': '新年', 't': 99}]

In [205]:
sql_4="select (@id:=@id+1) as id,title,count(`id`) as t from weiboevents,(SELECT @id:=0 )as id_temp group by title"
db = MysqlUtil()
count_number = db.fetchall(sql_4)

In [206]:
for i in range(len(count_number)):
    count_number[i]["id"]=int(count_number[i]["id"])



In [207]:
 count_number

[{'id': 1, 'title': '张新成 受伤', 't': 198}, {'id': 2, 'title': '新年', 't': 99}]